# Access an ASAM ODS EXD-API Plugin

## Prepare Python Environment to Access GRPC Service

In [1]:
try:
    from odsbox.proto import ods_external_data_pb2 as exd
except:
    # first time you need to install dependencies
    !python -m pip install --upgrade pip
    !python -m pip install -U odsbox[exd-data]



   ---------------------------------------- 0/6 [urllib3]
   ---------------------------------------- 0/6 [urllib3]
   ---------------------------------------- 0/6 [urllib3]
   ---------------------------------------- 0/6 [urllib3]
   ---------------------------------------- 0/6 [urllib3]
   ---------------------------------------- 0/6 [urllib3]
   ------ --------------------------------- 1/6 [idna]
   ------ --------------------------------- 1/6 [idna]
   ------ --------------------------------- 1/6 [idna]
   ------------- -------------------------- 2/6 [charset-normalizer]
   ------------- -------------------------- 2/6 [charset-normalizer]
   -------------------- ------------------- 3/6 [certifi]
   -------------------------- ------------- 4/6 [requests]
   -------------------------- ------------- 4/6 [requests]
   --------------------------------- ------ 5/6 [odsbox]
   --------------------------------- ------ 5/6 [odsbox]
   ---------------------------------------- 6/6 [odsbox]



In [2]:
import os
import pathlib

import grpc
from google.protobuf.json_format import MessageToJson

from odsbox.proto import ods_pb2
from odsbox.proto import ods_external_data_pb2 as exd
from odsbox.proto import ods_external_data_pb2_grpc as exd_grpc

## EXD-API

The EXD-API plugin is running as a RPC service at a given URL.
Running `exd_api_server.py`´will run the plugin at the given URL.

In [3]:
exd_api_plugin_url = "localhost:50051"

## Import Phase

We will open a file using the EXD-API and extract the internal structure of the file to import it into the ASAM ODS server.

In [4]:
data_file_path = os.path.abspath('data/example.csv')
if not os.path.exists(data_file_path):
    raise Exception('Data file is missing')

In [5]:
import_file_url = pathlib.Path(data_file_path).as_uri()
import_file_parameters=""
print(import_file_url)

# Will be filled from Structure
access_file_url = None
access_file_parameters = None

file:///c:/Users/andre/github/asam_ods_exd_api_pandascsv/data/example.csv


### Extract Infos from Structure

The structure contains infos about groups and channels to create corresponding measurements, submatrices and measurement_quantities

In [6]:
with grpc.insecure_channel(exd_api_plugin_url) as channel:
    stub = exd_grpc.ExternalDataReaderStub(channel)

    # import file into ASAM ODS Server physical storage
    import_identifier = exd.Identifier(
        url=import_file_url,
        parameters=import_file_parameters)

    import_handle = stub.Open(import_identifier)
    try:
        structure = stub.GetStructure(
            exd.StructureRequest(
                handle=import_handle))
        print(MessageToJson(structure))

        access_file_url = structure.identifier.url
        access_file_parameters = structure.identifier.parameters

        for group in structure.groups:
            group_id = group.id
            for channel in group.channels:
                channel_id = channel.id
    finally:
        stub.Close(import_handle)

{
  "identifier": {
    "url": "file:///c:/Users/andre/github/asam_ods_exd_api_pandascsv/data/example.csv"
  },
  "name": "example.csv",
  "groups": [
    {
      "name": "data",
      "totalNumberOfChannels": "3",
      "numberOfRows": "3",
      "channels": [
        {
          "name": "a",
          "dataType": "DT_LONGLONG"
        },
        {
          "id": "1",
          "name": "b",
          "dataType": "DT_DOUBLE"
        },
        {
          "id": "2",
          "name": "c",
          "dataType": "DT_DOUBLE"
        }
      ]
    }
  ]
}


## Access Bulk Data

With the stored information the ASAM ODS server can access the bulk data from the EXD-API plugin

In [7]:
with grpc.insecure_channel(exd_api_plugin_url) as channel:
    stub = exd_grpc.ExternalDataReaderStub(channel)

    # info from physical storage
    access_group_id = 0
    access_channel_ids = [0, 1]
    access_identifier = exd.Identifier(
        url=access_file_url,
        parameters=access_file_parameters)

    # open bulk access
    access_handle = stub.Open(access_identifier)
    try:
        request = exd.ValuesRequest(
            handle=access_handle,
            group_id=access_group_id,
            channel_ids=access_channel_ids)

        # read first chunk
        request.start = 0
        request.limit = 3
        values = stub.GetValues(request)
        print(MessageToJson(values))

    finally:
        stub.Close(access_handle)

{
  "channels": [
    {
      "values": {
        "dataType": "DT_LONGLONG",
        "longlongArray": {
          "values": [
            "1",
            "2",
            "3"
          ]
        }
      }
    },
    {
      "id": "1",
      "values": {
        "dataType": "DT_DOUBLE",
        "doubleArray": {
          "values": [
            2.1,
            2.2,
            2.3
          ]
        }
      }
    }
  ]
}
